# Simple Neural Network

## Task 4
In this task you have to implement a neural network from scratch. And just like in the exercise before, where you had to implement a perceptron, you again have to tackle this task without the utilization of any frameworks like TensorFlow or PyTorch.  
The goal of this task is to verify your manually calculated results, thus you should print out the necessary information during the training process of the network. Besides that your class should implement the following methods:
- init()
- fit()
- forward()/predict()
- backward()
- evaluate()

Additionally your implementation should display the epoch number, accuracy and further parameters of your choice during the training process.

#### Implementation
Imports here:

In [ ]:
# Imports
import numpy as np 

Implement your class representing the neural network and all of its previously mentioned methods here:  

In [ ]:

##########################
### MODEL
##########################

def sigmoid(z):
    return 1. / (1. + np.exp(-z))


def int_to_onehot(y, num_labels):

    ary = np.zeros((y.shape[0], num_labels))
    for i, val in enumerate(y):
        ary[i, val] = 1

    return ary


class NeuralNetMLP:

    def __init__(self, num_features, num_hidden, num_classes, learning_rate=0.1, random_seed=123):
        super().__init__()

        self.num_classes = num_classes
        self.learning_rate = learning_rate

        # hidden
        # rng = np.random.RandomState(random_seed)

        # self.weight_h = rng.normal(
        #     loc=0.0, scale=0.1, size=(num_hidden, num_features))
        # self.weight_h = np.array([[0.3,0.1],[0.7,0.5]])
        self.weight_h = np.array([[0.3,0.7],[-0.1,0.5]])
        self.bias_h = np.zeros(num_hidden)
        self.bias_h = np.array([0.4,0.4])
        # # output
        # self.weight_out = rng.normal(
        #     loc=0.0, scale=0.1, size=(num_classes, num_hidden))
        # self.bias_out = np.zeros(num_classes)
        self.weight_out = np.array([[0.6,-0.7],[0.3,0.8]])
        self.bias_out = np.array([0.15,0.15])

    def forward(self, x):
        # Hidden layer
        # input dim: [n_examples, n_features] dot [n_hidden, n_features].T
        # output dim: [n_examples, n_hidden]
        z_h = np.dot(x, self.weight_h.T) + self.bias_h
        a_h = sigmoid(z_h)
        print(f"z_h: {z_h}  a_h: {a_h}")
        # Output layer
        # input dim: [n_examples, n_hidden] dot [n_classes, n_hidden].T
        # output dim: [n_examples, n_classes]
        z_out = np.dot(a_h, self.weight_out.T) + self.bias_out
        a_out = sigmoid(z_out)
        print(f"z_out: {z_out}  a_out: {a_out}")
        return a_h, a_out

    def backward(self, x, a_h, a_out, y):  
    
        #########################
        ### Output layer weights
        #########################
        
        # onehot encoding
        # y_onehot = int_to_onehot(y, self.num_classes)
        y_onehot = np.array(y)
        # Part 1: dLoss/dOutWeights
        ## = dLoss/dOutAct * dOutAct/dOutNet * dOutNet/dOutWeight
        ## where DeltaOut = dLoss/dOutAct * dOutAct/dOutNet
        ## for convenient re-use
        
        # input/output dim: [n_examples, n_classes]
        d_loss__d_a_out = 2.*(a_out - y_onehot) / y.shape[0]
        d_loss__d_a_out = (a_out - y_onehot)
        print(f"d_loss__d_a_out: {d_loss__d_a_out}")
        print(y_onehot)
        # input/output dim: [n_examples, n_classes]
        d_a_out__d_z_out = a_out * (1. - a_out) # sigmoid derivative
        print(f"d_a_out__d_z_out: {d_a_out__d_z_out}")
        # output dim: [n_examples, n_classes]
        delta_out = d_loss__d_a_out * d_a_out__d_z_out # "delta (rule) placeholder"
        # print(f"delta_out: {delta_out}")
        # gradient for output weights
        
        # [n_examples, n_hidden]
        d_z_out__dw_out = a_h
        
        # input dim: [n_classes, n_examples] dot [n_examples, n_hidden]
        # output dim: [n_classes, n_hidden]
        print(f"delta_out.T: {delta_out.T}  d_z_out__dw_out: {d_z_out__dw_out}")
        d_loss__dw_out = np.dot(delta_out.T, d_z_out__dw_out)
        print(f"d_loss__dw_out: {d_loss__dw_out}")
        d_loss__db_out = np.sum(delta_out, axis=0)
        print(f"d_loss__db_out: {d_loss__db_out}")
        

        #################################        
        # Part 2: dLoss/dHiddenWeights
        ## = DeltaOut * dOutNet/dHiddenAct * dHiddenAct/dHiddenNet * dHiddenNet/dWeight
        
        # [n_classes, n_hidden]
        d_z_out__a_h = self.weight_out
        
        # output dim: [n_examples, n_hidden]
        d_loss__a_h = np.dot(delta_out, d_z_out__a_h)
        
        # [n_examples, n_hidden]
        d_a_h__d_z_h = a_h * (1. - a_h) # sigmoid derivative
        
        # [n_examples, n_features]
        d_z_h__d_w_h = x
        
        # output dim: [n_hidden, n_features]
        d_loss__d_w_h = np.dot((d_loss__a_h * d_a_h__d_z_h).T, d_z_h__d_w_h)
        d_loss__d_b_h = np.sum((d_loss__a_h * d_a_h__d_z_h), axis=0)

        return d_loss__dw_out, d_loss__db_out, d_loss__d_w_h, d_loss__d_b_h

    def get_accuracy(self, y_correct, y_preds):
        return np.mean(y_correct == y_preds)
    
    def train(self,x,y,epochs):
        y = np.array(y)
        for epoch in range(epochs):
            ah, aout = self.forward(x)
            #print(f"ah: {ah}   aout: {aout}")
            d_loss__dw_out, d_loss__db_out, d_loss__d_w_h, d_loss__d_b_h = self.backward(x,ah,aout,y)
            #print(f"d_loss__dw_out: {d_loss__dw_out} d_loss__db_out: {d_loss__db_out} ")
            self.weight_h -= self.learning_rate * d_loss__d_w_h
            self.bias_h -= self.learning_rate * d_loss__d_b_h
            self.weight_out -= self.learning_rate * d_loss__dw_out
            self.bias_out -= self.learning_rate * d_loss__db_out

            print(f"self.weight_h: {self.weight_h}   self.weight_out: {self.weight_out}  acc:{self.get_accuracy(y,aout)} ah: {ah}   aout: {aout}")

            

Define input and target lists here. Create an instance of your network class and use it to perform at least one training step. Does the output match with your calculations?

In [ ]:
x = [0.2,0.6]
y = [1,0]

nn = NeuralNetMLP(2,2,2,0.1)

ah, aout = nn.forward(x)
nn.backward(x,ah,aout,np.array(y))
nn.train(x,y,1000)